In [9]:
swat_freq_list = ['P1_FIT101', 'P1_LIT101', 'P1_MV101', 'P1_P101', 'P1_P102', 'P2_AIT201',
       'P2_AIT202', 'P2_AIT203', 'P2_FIT201', 'P2_MV201', 'P2_P201', 'P2_P202',
       'P2_P203', 'P2_P204', 'P2_P205', 'P2_P206', 'P3_DPIT301', 'P3_FIT301',
       'P3_LIT301', 'P3_MV301', 'P3_MV302', 'P3_MV303', 'P3_MV304', 'P3_P301',
       'P3_P302', 'P4_AIT401', 'P4_AIT402', 'P4_FIT401', 'P4_LIT401',
       'P4_P401', 'P4_P402', 'P4_P403', 'P4_P404', 'P4_UV401', 'P5_AIT501',
       'P5_AIT502', 'P5_AIT503', 'P5_AIT504', 'P5_FIT50 1', 'P5_FIT502',
       'P5_FIT503', 'P5_FIT504', 'P5_P501', 'P5_P502', 'P5_PIT501',
       'P5_PIT502', 'P5_PIT503', 'P6_FIT601', 'P6_P601', 'P6_P602', 'P6_P603']

In [1]:
import warnings
warnings.filterwarnings('ignore')

from dataset import *

from cluster import *
from classifier import *
from preprocessing import *
import numpy as np

from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import logging
import time
import datetime
from sklearn import svm
from matplotlib import pyplot as plt
import argparse

from scipy import signal
from tapr import *
from util import *

from itertools import combinations
import pdb

In [2]:
np.random.seed(777)

parser = argparse.ArgumentParser()
parser.add_argument('--cov_type', type = str, default = 'full')
parser.add_argument('--gamma', type = float, default = 0.1)
parser.add_argument('--C', type = float, default = 1000)
parser.add_argument('--exp', type = str, default = 'tp')
parser.add_argument('--selected_dim', nargs='+', type=int, default=[[0,1],[2,3],[3,1,2]])
parser.add_argument('--swat_freq_select_list', nargs='+', type=str, default=[['2_FIT_002_PV']])
parser.add_argument('--read_size', type = int, default = 5)
args = parser.parse_args([])    

delta = 600
theta = 0.001
alpha = 1.0
label = [0, 1]
ev = TaPR(label, theta, delta)

In [12]:
args.swat_freq_select_list = ['P1_LIT101', 'P1_MV101', 'P1_P101' ,'P2_P203', 'P3_DPIT301', 'P3_LIT301','P3_MV301','P4_LIT401', 'P5_AIT503']
now = datetime.datetime.now()
today = '%s-%s-%s'%(now.year, now.month, now.day)
args.exp = 'swat 최종발표 준비'
tap_path = './prediction_' + today + args.exp + '.csv'

swat_dic = dict()
swat_dic['data_path'] = './swat_data/'

raw_selected_dim = [0,1,2,38,39,40]
freq_selected_dim = [0,1,31,32]

swat_n_cluster_list = list(np.arange(5,10))


window_size = 30

In [18]:
args.read_size = 1

In [19]:
train_x_swat, train_y_swat, val_x_swat, val_y_swat, test_x_swat, test_y_swat, _ = SWaT_dataset(data_path = swat_dic['data_path'], read_size = args.read_size)

train_x_swat_freq = get_freq_data_2(data = train_x_swat, 
                freq_select_list = args.swat_freq_select_list, read_size = args.read_size,
                                      window_size = window_size)
val_x_swat_freq = get_freq_data_2(data = val_x_swat, 
                freq_select_list = args.swat_freq_select_list, read_size = args.read_size,
                                      window_size = window_size)
test_x_swat_freq = get_freq_data_2(data = test_x_swat, 
                freq_select_list = args.swat_freq_select_list, read_size = args.read_size,
                                      window_size = window_size)



train_x_modify_swat, val_x_modify_swat, test_x_modify_swat = PCA_preprocessing_modify(scaler = 'standard', \
                                                                train_x = train_x_swat, val_x = val_x_swat, \
                                                              test_x= test_x_swat, n_neighbors = 4, 
                                                              n_components = 2, feature_num =2, \
                                                              selected_dim= raw_selected_dim)


# train_x_modify_swat_freq, val_x_modify_swat_freq, test_x_modify_swat_freq = PCA_preprocessing_modify(scaler = 'standard', \
#                                                                 train_x = train_x_swat_freq, val_x = val_x_swat_freq, \
#                                                               test_x= test_x_swat_freq, n_neighbors = 4, 
#                                                               n_components = 2, feature_num =2, \
#                                                               selected_dim= freq_selected_dim)

# train_x_modify_swat = np.concatenate((train_x_modify_swat, train_x_modify_swat_freq),1)
# val_x_modify_swat = np.concatenate((val_x_modify_swat, val_x_modify_swat_freq),1)
# test_x_modify_swat = np.concatenate((test_x_modify_swat, test_x_modify_swat_freq),1)

train_x_modify_swat = np.concatenate((train_x_modify_swat, train_x_swat_freq),1)
val_x_modify_swat = np.concatenate((val_x_modify_swat, val_x_swat_freq),1)
test_x_modify_swat = np.concatenate((test_x_modify_swat, test_x_swat_freq),1)

In [22]:
optimal_f1_swat_list = []
optimal_tar_swat_list = []
optimal_tap_swat_list = []
optimal_cluster_combi_swat_list = []

for swat_n_clusters in swat_n_cluster_list:
    print("n_clutser {}".format(swat_n_clusters))

    train_predict_swat_list = []
    test_predict_swat_list = []
    label_for_train_swat_list = []
    label_for_var_swat_list = []
    label_for_test_swat_list = []
    classifier_swat_list = []

    tp_optimal_f1_swat = 0
    tp_optimal_tar_swat = 0
    tp_optimal_tap_swat = 0
    tp_optimal_cluster_combi_swat = 0


    train_label_swat, val_label_swat, test_label_swat, cluster_model_gmm_swat = GaussianMixture_clustering(
        n_clusters=swat_n_clusters,train_x = train_x_modify_swat,val_x= val_x_modify_swat,
        test_x = test_x_modify_swat,covariance_type = 'tied') 

    for cluster_num in range(np.unique(train_label_swat)[-1] + 1):
        label_for_train_swat = (train_label_swat == cluster_num).astype(int)
        label_for_var_swat = (val_label_swat == cluster_num).astype(int)
        label_for_test_swat = (test_label_swat == cluster_num).astype(int)
        if np.unique(label_for_train_swat).shape[0] == 1:
            label_for_train_swat[0] = 1
            label_for_test_swat[0] = 1
            label_for_var_swat[0] = 1


        label_for_train_swat_list.append(label_for_train_swat)
        label_for_var_swat_list.append(label_for_var_swat)
        label_for_test_swat_list.append(label_for_test_swat)

        classifier_swat = svm.SVC(gamma = args.gamma, C = args.C)
        classifier_swat.fit(train_x_modify_swat, label_for_train_swat)
        classifier_swat_list.append(classifier_swat)

        pred_train_swat = classifier_swat.predict(train_x_modify_swat)
        pred_test_swat = classifier_swat.predict(test_x_modify_swat)

        test_predict_swat_list.append(pred_test_swat)
        train_predict_swat_list.append(pred_test_swat)

    # swat tapr modified version.
    ev.load_anomalies( './swat_label.csv')

    for combi_num in range(2, swat_n_clusters + 1):
        swat_combi_list = list(combinations(test_predict_swat_list, combi_num))
        combi_list = list(combinations(np.arange(swat_n_clusters), combi_num))

        swat_f_tapr_beta = 0
        swat_f1 = 0
        swat_j = 0
        swat_combi_max = 0

        swat_list = 0
        swat_pr = 0
        swat_re = 0

        swat_tar = 0
        swat_tap = 0

        swat_alpha = 0
        undetected = 0

        swat_j = 0

        for combi_ in range(len(swat_combi_list)):
            swat_predict = 0

            for i in range(len(swat_combi_list[combi_])):
                swat_predict += swat_combi_list[combi_][i]

            for j in range(1, len(swat_combi_list[combi_]) -1):

                pd.DataFrame((swat_predict < j).astype(int)).to_csv(tap_path,index=False, header=None)
                swat_list_tp = (swat_predict < j).astype(int)

                ev.load_predictions(tap_path)

                tapd_value, _ = ev.TaP_d()
                tard_value, _ = ev.TaR_d()
                tapp_value = ev.TaP_p()            
                tarp_value = ev.TaR_p()

                tar = 0.5*tard_value + 0.5*tarp_value
                tap = 0.5*tapd_value + 0.5*tapp_value
                tp_f1 = f1_score(test_y_swat, (swat_predict < j).astype(int))
                tp_pr = precision_score(test_y_swat, (swat_predict < j).astype(int))
                tp_re = recall_score(test_y_swat, (swat_predict < j).astype(int))

                beta = 1
                tp_f_tapr_beta = (1+(beta**2))*(tap*tar)/((beta**2)*tap + tar+ 0.0001)

                if  tp_f1 > swat_f1:
                    swat_f_tapr_beta = tp_f_tapr_beta
                    swat_tap = tap
                    swat_tar = tar
                    swat_f1 = tp_f1
                    swat_pr = tp_pr
                    swat_re = tp_re

                    swat_combi_max = combi_list[combi_]

                    swat_list = swat_list_tp
                    undetected = find_attack_num(swat_list, test_y_swat)
                    swat_j = j

        print("-"*30) 
        print('swat pr {:.4f}, re{:.4f}, f1 {:.4f}, combi_list {}, undetectd {} j {}'.\
              format(swat_pr, swat_re ,swat_f1, swat_combi_max, undetected, swat_j))
        print("f tarp {:.4f} tar {:.4f} tap {:.4f}".format(swat_f_tapr_beta, swat_tar, swat_tap))
        print(np.unique(swat_list, return_counts=True))
        print("-"*30) 
        if tp_optimal_f1_swat < swat_f1:
            tp_optimal_f1_swat = swat_f1
            tp_optimal_tar_swat = swat_tar
            tp_optimal_tap_swat = swat_tap
            tp_optimal_cluster_combi_swat = swat_combi_max

    optimal_f1_swat_list.append(tp_optimal_f1_swat)
    optimal_tar_swat_list.append(tp_optimal_tar_swat)
    optimal_tap_swat_list.append(tp_optimal_tap_swat)
    optimal_cluster_combi_swat_list.append(tp_optimal_cluster_combi_swat)      


print(optimal_f1_swat_list)
print(optimal_tar_swat_list)
print(optimal_tap_swat_list)
print(optimal_cluster_combi_swat_list)


print("-"*30)
print("-"*30)
print('FINISH')   
print('%s:%s:%s'%(datetime.now().hour, datetime.now().minute, datetime.now().second))

n_clutser 5
------------------------------
swat pr 0.0000, re0.0000, f1 0.0000, combi_list 0, undetectd 0 j 0
f tarp 0.0000 tar 0.0000 tap 0.0000
(array([0]), array([1]))
------------------------------
------------------------------
swat pr 0.1214, re1.0000, f1 0.2165, combi_list (1, 2, 3), undetectd 0 j 1
f tarp 0.0000 tar 0.0000 tap 0.0000
(array([1]), array([449919]))
------------------------------
------------------------------
swat pr 0.1214, re1.0000, f1 0.2165, combi_list (0, 1, 2, 3), undetectd 0 j 2
f tarp 0.0000 tar 0.0000 tap 0.0000
(array([1]), array([449919]))
------------------------------
------------------------------
swat pr 0.1214, re1.0000, f1 0.2165, combi_list (0, 1, 2, 3, 4), undetectd 0 j 2
f tarp 0.0000 tar 0.0000 tap 0.0000
(array([1]), array([449919]))
------------------------------
n_clutser 6
------------------------------
swat pr 0.0000, re0.0000, f1 0.0000, combi_list 0, undetectd 0 j 0
f tarp 0.0000 tar 0.0000 tap 0.0000
(array([0]), array([1]))
---------

AttributeError: module 'datetime' has no attribute 'now'

현재 보니 그냥 넣는것은 성능이 매우 떨어질 뿐더러 매우 느리다. 